### Relax Data Science Challenge

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
users = pd.read_csv('takehome_users.csv', encoding='ISO-8859-1', index_col='object_id')
engagement = pd.read_csv('takehome_user_engagement.csv')

In [3]:
users.head()

creation_time               name                       email  \
object_id                                                                       
1          2014-04-22 03:53:30     Clausen August    AugustCClausen@yahoo.com   
2          2013-11-15 03:45:04      Poole Matthew      MatthewPoole@gustr.com   
3          2013-03-19 23:14:52  Bottrill Mitchell  MitchellBottrill@gustr.com   
4          2013-05-21 08:09:28    Clausen Nicklas   NicklasSClausen@yahoo.com   
5          2013-01-17 10:14:20          Raw Grace          GraceRaw@yahoo.com   

          creation_source  last_session_creation_time  \
object_id                                               
1            GUEST_INVITE                1.398139e+09   
2              ORG_INVITE                1.396238e+09   
3              ORG_INVITE                1.363735e+09   
4            GUEST_INVITE                1.369210e+09   
5            GUEST_INVITE                1.358850e+09   

           opted_in_to_mailing_list  enabled_for_marketing_drip  org_id  \
object_id                                                                 
1                                 1                           0      11   
2                                 0                           0       1   
3                                 0                           0      94   
4                                 0                           0       1   
5                                 0                           0     193   

           invited_by_user_id  
object_id                      
1                     10803.0  
2                       316.0  
3                      1525.0  
4                      5151.0  
5                      5240.0

In [4]:
engagement.head()

time_stamp  user_id  visited
0  2014-04-22 03:53:30        1        1
1  2013-11-15 03:45:04        2        1
2  2013-11-29 03:45:04        2        1
3  2013-12-09 03:45:04        2        1
4  2013-12-25 03:45:04        2        1

In [5]:
engagement.time_stamp = pd.to_datetime(engagement.time_stamp)
users.creation_time = pd.to_datetime(users.creation_time)

In [6]:
engagement.head()

time_stamp  user_id  visited
0 2014-04-22 03:53:30        1        1
1 2013-11-15 03:45:04        2        1
2 2013-11-29 03:45:04        2        1
3 2013-12-09 03:45:04        2        1
4 2013-12-25 03:45:04        2        1

In [7]:
# Set time_stamp column as index
#next_7days = [engagement.time_stamp + datetime.timedelta(days=x) for x in range(0,7)]
#engagement['visit_next_7days'] = engagement.time_stamp.map((lambda x: (x - datetime.timedelta(days=y) for y in range(0,7))))

In [22]:
engagement.head()

time_stamp  user_id  visited
0 2014-04-22 03:53:30        1        1
1 2013-11-15 03:45:04        2        1
2 2013-11-29 03:45:04        2        1
3 2013-12-09 03:45:04        2        1
4 2013-12-25 03:45:04        2        1

In [9]:
eng2 = engagement.groupby('user_id')

In [10]:
def check(x):

    if len(x) >= 3:  # condition to eliminate if user has less than 3 time stamps
        x = [i for i in x]  # convert property object to list
        x.sort()  # sort the dates in increasing order
        x = [x[i+1] - x[i] for i in range(len(x)-2)]  # compute cumulative difference of current and next day
        # sum i, i+1, i+2 terms and check if it`s less than 7 which gives if it`s in a seven day period
        x = [1 for i in range(len(x)-2) if x[i] + x[i+1] + x[i+2] <= timedelta(days=7)]
        # condition to check if there`s a 1 in x, return 1 if true
        if 1 in x:
            return 1

In [11]:
eng3 = engagement.groupby('user_id').agg(check)

In [12]:
eng3.head(10)

time_stamp
user_id            
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
10              1.0
11              NaN
13              NaN

In [13]:
eng3.fillna(0, inplace = True)
eng3.columns = ['adopted']
eng3.head(10)


adopted
user_id         
1            0.0
2            0.0
3            0.0
4            0.0
5            0.0
6            0.0
7            0.0
10           1.0
11           0.0
13           0.0

In [14]:
df = users.join(eng3, how='left')
df.head()

creation_time               name                       email  \
object_id                                                                      
1         2014-04-22 03:53:30     Clausen August    AugustCClausen@yahoo.com   
2         2013-11-15 03:45:04      Poole Matthew      MatthewPoole@gustr.com   
3         2013-03-19 23:14:52  Bottrill Mitchell  MitchellBottrill@gustr.com   
4         2013-05-21 08:09:28    Clausen Nicklas   NicklasSClausen@yahoo.com   
5         2013-01-17 10:14:20          Raw Grace          GraceRaw@yahoo.com   

          creation_source  last_session_creation_time  \
object_id                                               
1            GUEST_INVITE                1.398139e+09   
2              ORG_INVITE                1.396238e+09   
3              ORG_INVITE                1.363735e+09   
4            GUEST_INVITE                1.369210e+09   
5            GUEST_INVITE                1.358850e+09   

           opted_in_to_mailing_list  enabled_for_marketing_drip  org_id  \
object_id                                                                 
1                                 1                           0      11   
2                                 0                           0       1   
3                                 0                           0      94   
4                                 0                           0       1   
5                                 0                           0     193   

           invited_by_user_id  adopted  
object_id                               
1                     10803.0      0.0  
2                       316.0      0.0  
3                      1525.0      0.0  
4                      5151.0      0.0  
5                      5240.0      0.0

### EDA

In [15]:
# Drop name and email
df.drop(['name', 'email', 'creation_time', 'last_session_creation_time'], inplace = True, axis = 1)

In [16]:
# We need to encode the categorical features as binary variables to use them in sklearn’s random forest classifier.
X_original = df
X_original = pd.get_dummies(X_original)
X_original.head()

opted_in_to_mailing_list  enabled_for_marketing_drip  org_id  \
object_id                                                                 
1                                 1                           0      11   
2                                 0                           0       1   
3                                 0                           0      94   
4                                 0                           0       1   
5                                 0                           0     193   

           invited_by_user_id  adopted  creation_source_GUEST_INVITE  \
object_id                                                              
1                     10803.0      0.0                             1   
2                       316.0      0.0                             0   
3                      1525.0      0.0                             0   
4                      5151.0      0.0                             1   
5                      5240.0      0.0                             1   

           creation_source_ORG_INVITE  creation_source_PERSONAL_PROJECTS  \
object_id                                                                  
1                                   0                                  0   
2                                   1                                  0   
3                                   1                                  0   
4                                   0                                  0   
5                                   0                                  0   

           creation_source_SIGNUP  creation_source_SIGNUP_GOOGLE_AUTH  
object_id                                                              
1                               0                                   0  
2                               0                                   0  
3                               0                                   0  
4                               0                                   0  
5                               0                                   0

In [17]:
X_original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
opted_in_to_mailing_list              12000 non-null int64
enabled_for_marketing_drip            12000 non-null int64
org_id                                12000 non-null int64
invited_by_user_id                    6417 non-null float64
adopted                               8823 non-null float64
creation_source_GUEST_INVITE          12000 non-null uint8
creation_source_ORG_INVITE            12000 non-null uint8
creation_source_PERSONAL_PROJECTS     12000 non-null uint8
creation_source_SIGNUP                12000 non-null uint8
creation_source_SIGNUP_GOOGLE_AUTH    12000 non-null uint8
dtypes: float64(2), int64(3), uint8(5)
memory usage: 941.1 KB


In [18]:
# fill '0' if not invited by anyone
X_original['invited_by_user_id'].fillna(0, inplace=True)
X_original.dropna(axis=0, inplace=True)

### Machine Learning

In [19]:
# Split data into train and test:
from sklearn.model_selection import train_test_split
X = X_original.drop('adopted', axis=1)
Y = X_original['adopted']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25, random_state=42)

#### Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, precision_score, precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report
clf_rf = RandomForestClassifier(random_state=12)
param_grid = { 
    'n_estimators': [20, 50, 100]
    #'max_features': ['auto', 'sqrt', 'log2'],
   # 'criterion' :['gini', 'entropy']
}

In [21]:
CV_rfc = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv= 5)
CV_rfc.fit(X, Y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [23]:
CV_rfc.best_params_

{'n_estimators': 50}

In [24]:
CV_rfc.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [25]:
CV_rfc.best_score_

0.80267482715629601

In [26]:
clf_rf2 = CV_rfc.best_estimator_
clf_rf2.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=12, verbose=0, warm_start=False)

In [27]:
Y_pred = clf_rf2.predict(Xtest)
precision = precision_score(Ytest, Y_pred)
recall = recall_score(Ytest, Y_pred)
print("Precison score: ", precision)
print("Recall score: ", recall)
print(confusion_matrix(Ytest, Y_pred))
print(classification_report(Ytest, Y_pred))

Precison score:  0.894514767932
Recall score:  0.605714285714
[[1831   25]
 [ 138  212]]
             precision    recall  f1-score   support

        0.0       0.93      0.99      0.96      1856
        1.0       0.89      0.61      0.72       350

avg / total       0.92      0.93      0.92      2206



#### What are the most important features?

In [28]:
features_df = pd.DataFrame(clf_rf2.feature_importances_, index = X.columns)
features_df.sort_values(by=0,ascending=False)

0
org_id                              0.612864
invited_by_user_id                  0.346665
opted_in_to_mailing_list            0.013734
enabled_for_marketing_drip          0.012739
creation_source_GUEST_INVITE        0.004134
creation_source_ORG_INVITE          0.003164
creation_source_PERSONAL_PROJECTS   0.002303
creation_source_SIGNUP              0.002245
creation_source_SIGNUP_GOOGLE_AUTH  0.002152